In [138]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm

import re
import codecs
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords 

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda, Dropout, Bidirectional, Dense
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras


In [34]:
data1 = './ag_news_csv/train.csv'
data2 = './dbpedia_csv/train.csv'
data3 = './News_Category_Dataset_v2.json'

In [35]:
data3 = pd.read_json(data3, lines=True)
# data3.head()

In [36]:
data3 = data3.drop(columns = ['authors', 'link', 'short_description', 'date'])
# data3.head()

In [37]:
deleteSections = ['BLACK VOICES', 'DIVORCE', 'FIFTY', 'LATINO VOICES', 'MEDIA', 'QUEER VOICES', 'GOOD NEWS', 'IMPACT', "WEDDINGS", "WEIRD NEWS", "MONEY", 'PARENTING', 'PARENTS', "WOMEN", "GREEN", "ENVIRONMENT", "COMEDY"]
replaces = {
    "ARTS": "ARTS & CULTURE",
    "CULTURE & ARTS": "ARTS & CULTURE",
    "FOOD & DRINK": "ENTERTAINMENT",
    "HEALTHY LIVING": "HOME & LIVING",
    "THE WORLDPOST": "WORLD NEWS",
    "WORLDPOST": "WORLD NEWS",
    "STYLE": "STYLE & BEAUTY",
    "TASTE": "STYLE & BEAUTY",
    "WELLNESS" : "STYLE & BEAUTY",
    "COLLEGE":"EDUCATION",
    "TECH" : "SCIENCE & TECH",
    "SCIENCE" : "SCIENCE & TECH",
}

In [38]:
# data3.drop(data3[data3.category < 50].index, inplace=True)
deleteRows = []
replacesKeys = replaces.keys()
for values in tqdm(range(len(data3.category))):
    if deleteSections.count(data3.category[values]) != 0:
        deleteRows.append(values)
    elif data3.category[values] in replacesKeys:
        data3.category[values] = replaces.get(data3.category[values])
data3clean = data3.copy()

100%|██████████| 200853/200853 [02:48<00:00, 1189.67it/s]


In [39]:
print(len(deleteRows))
print(np.shape(data3))
data3 = data3.drop(labels = deleteRows, axis = 0)
np.shape(data3)

57740
(200853, 2)


(143113, 2)

In [40]:
glove = './glove.6B.100d'

def load_glove_model(glove_file):

    f = codecs.open(glove_file + ".txt", 'r', encoding='utf-8')
    model = {}
    for line in f:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        model[word] = embedding
    return model

#build the model-> a bit slow ~30seconds 
model = load_glove_model(glove)

In [126]:
def get_w2v(sentence, model):
    return np.array([model.get(val, np.zeros(100)) for val in sentence.split()], dtype=np.float64)

def preprocess(input):
    if type(input) != type("string"):
        return "skip" 
    input = input.lower()
    input = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", input)
    input = re.sub(r"what's", "what is ", input)
    input = re.sub(r"\'s", " ", input)
    input = re.sub(r"\'ve", " have ", input)
    input = re.sub(r"can't", "cannot ", input)
    input = re.sub(r"n't", " not ", input)
    input = re.sub(r"i'm", "i am ", input)
    input = re.sub(r"\'re", " are ", input)
    input = re.sub(r"\'d", " would ", input)
    input = re.sub(r"\'ll", " will ", input)
    input = re.sub(r",", " ", input)
    input = re.sub(r"\.", " ", input)
    input = re.sub(r"!", " ! ", input)
    input = re.sub(r"\/", " ", input)
    input = re.sub(r"\^", " ^ ", input)
    input = re.sub(r"\+", " + ", input)
    input = re.sub(r"\-", " - ", input)
    input = re.sub(r"\=", " = ", input)
    input = re.sub(r"'", " ", input)
    input = re.sub(r"(\d+)(k)", r"\g<1>000", input)
    input = re.sub(r":", " : ", input)
    input = re.sub(r" e g ", " eg ", input)
    input = re.sub(r" b g ", " bg ", input)
    input = re.sub(r" u s ", " american ", input)
    input = re.sub(r"\0s", "0", input)
    input = re.sub(r" 9 11 ", "911", input)
    input = re.sub(r"e - mail", "email", input)DataFrame
    input = re.sub(r"j k", "jk", input)
    input = re.sub(r"\s{2,}", " ", input)
    return input

def padZeros(input, maxLen):
    if len(input) >= maxlen or len(input) < 4:
        return "skip"
    if np.shape(input)[0] == maxLen:
        return input
    lenToPad = maxLen - np.shape(input)[0]
    output = np.concatenate((input, np.zeros((lenToPad, 100))))
    return output
    return np.transpose(output)

In [127]:
maxlen = 25
data3clean = data3.copy()
rowsToDelete = []
X = []
for values in tqdm(range(len(data3clean.headline))):
# for values in tqdm(range(0,6)):
    processedText = preprocess(data3clean.iloc[values, 1])
    # print(processedText)
    vectors = get_w2v(processedText, model)
    # print(vectors)
    padded = padZeros(vectors, maxlen)
    if padded == "skip":
        rowsToDelete.append(values)
    else:
        X = X + [padded]
        # data3clean.headline[values] = padded

100%|██████████| 143113/143113 [02:34<00:00, 926.71it/s]


In [130]:

np.shape(X)

(140197, 25, 100)

In [135]:
Y = pd.get_dummies(data3clean['category']).values.tolist()
Y = np.array(Y)

X = np.array(X)
Y = np.delete(Y, rowsToDelete, axis = 0)
print(np.shape(Y))
print(np.shape(X))
# data3clean = Y.drop(axis = 0, labels = rowsToDelete)
# np.shape(Y)


(140197, 13)
(140197, 25, 100)


In [ ]:
# data2 = './dbpedia_csv/train.csv'
# data2 = pd.read_csv(data2)
# data2.columns = ["category", "Headline", "Context"]
# classes = ["dummyVal", "WORLD NEWS", "SPORTS", "BUSINESS", "Sci/Tech"]
# for idx in tqdm(range(len(data2.category))):
#     data2.category[idx] = classes[data2.category[idx]]
# data2.head()

In [ ]:
#CONFUSION MATRIX TO SEE WHAT TO REMOVE

In [139]:
# Model variables
n_hidden = 80
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 8

# The visible layer
input = Input(shape=np.shape(X[0]), dtype='float32')
lstm = Bidirectional(LSTM(n_hidden, return_sequences=True))
lstm2 = LSTM(n_hidden)
output = Dense(13, activation = "softmax")

l1 = lstm(input)
l2 = Dropout(0.1)(l1)
l3 = lstm2(l2)
l4 = output(l3)
# Calculates the distance as defined by the MaLSTM model


# Pack it all up into a model
malstm = Model(input, l4)

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adam(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training

malstm_trained = malstm.fit(X, Y, batch_size=batch_size, nb_epoch=n_epoch, validation_split = 0.2)# Model variables


ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_3:0", shape=(None, 25, 100), dtype=float32) at layer "input_3". The following previous layers were accessed without issue: []